In [1]:
import pandas as pd

df = pd.read_csv("merged_df.csv")
df = pd.DataFrame(df)
df.head()

,Unnamed: 0,objectid,title,beginyear,endyear,medium,attribution,provenancetext,classification,subclassification,iiifthumburl,assistivetext
0,1,1,Saint Paul and a Group of Worshippers,1333.0,1333.0,tempera on panel,Bernardo Daddi,The often-repeated statement in the earlier li...,Painting,NaN,https://api.nga.gov/iiif/7bbcfd01-e774-46e7-96...,"Against a shiny gold background, a man with pe..."
1,2,17,Matteo Olivieri (?),1430.0,1440.0,tempera (and oil?) on panel transferred to canvas,Florentine 15th Century,"(Stefano Bardini [1836-1922], Florence); sold ...",Painting,NaN,https://api.nga.gov/iiif/8913742c-6155-4956-b6...,NaN
2,3,70,An Old Woman Dozing over a Book,1655.0,1655.0,oil on canvas,Nicolaes Maes,"Antoine Delacoux de Marivault [1771-1846], Par...",Painting,NaN,https://api.nga.gov/iiif/c5f5cfa6-6150-435a-bf...,Shown from the waist up on the other side of a...
3,4,25,Profile Portrait of a Lady,1410.0,1410.0,oil on panel,Franco-Flemish 15th Century,"Henry Valentine Stafford Jerningham, 9th Baron...",Painting,NaN,https://api.nga.gov/iiif/82e62b74-1316-4ed3-9b...,NaN
4,5,29,"The Crucifixion with the Virgin, Saint John, S...",1482.0,1485.0,oil on panel transferred to canvas,Pietro Perugino,Probably commissioned by Bartolommeo Bartoli (...,Painting,NaN,https://api.nga.gov/iiif/b43e331e-ede9-44ca-a2...,NaN


In [2]:
df = df.fillna('')
list_of_dicts = df.to_dict(orient='records')

list_of_dicts[:3]


[{'Unnamed: 0': 1,
  'objectid': 1,
  'title': 'Saint Paul and a Group of Worshippers',
  'beginyear': 1333.0,
  'endyear': 1333.0,
  'medium': 'tempera on panel',
  'attribution': 'Bernardo Daddi',
  'provenancetext': 'The often-repeated statement in the earlier literature that the panel comes from the Florentine monastery of San Felice in Piazza[1] does not seem to be based on any secure, or at any rate documented, evidence. Perhaps more plausible, based at least on the identity of the saint, is the more recent proposal of a provenance either from the Florentine Ospedale di San Paolo or from the nearby church of San Paolino, since a handwritten annotation on an old photograph indicated its provenance “dai padri di San Paolino.”[2] Elia Volpi [1858–1938], Florence, by the early 1900s;[3] (his sale, American Art Galleries, New York, 21-27 November 1916, seventh day, no. 1040, as “Primitive school of Tuscany, early XVth century”); (Bourgeois Galleries, New York);[4] purchased January 19

In [3]:
df.describe().round(2)

,Unnamed: 0,objectid
count,109576.00,109576.00
mean,57785.75,87573.21
std,33511.38,66784.27
min,1.00,1.00
25%,28565.75,29460.75
50%,58346.50,66717.50
75%,86213.25,145174.25
max,116240.00,228222.00


In [4]:
from spacy.lang.en import English 

nlp = English()

nlp.add_pipe("sentencizer")

doc = nlp("This is a sentence. This another sentence.")
assert len(list(doc.sents)) == 2

list(doc.sents)

[This is a sentence., This another sentence.]

In [5]:
import fitz 
from tqdm.auto import tqdm 
for item in tqdm(list_of_dicts):
    
    if(item["assistivetext"] == ""):
        item["sentences"] = list(nlp(item["provenancetext"]).sents)
    else:
        item["sentences"] = list(nlp(item["assistivetext"]).sents)
    
    # Make sure all sentences are strings
    item["sentences"] = [str(sentence) for sentence in item["sentences"]]
    
    # Count the sentences 
    item["page_sentence_count_spacy"] = len(item["sentences"])

  0%|          | 0/109576 [00:00<?, ?it/s]

In [6]:
import random

random.sample(list_of_dicts, k=1)

[{'Unnamed: 0': 40048,
  'objectid': 154488,
  'title': 'Doris McKinney, Republic Steel (Working People series)',
  'beginyear': 1978.0,
  'endyear': 1979.0,
  'medium': 'gelatin silver print',
  'attribution': 'Milton Rogovin',
  'provenancetext': 'Milton Rogovin, Buffalo, NY; David Knaus, Palm Springs, CA; (Morehouse Gallery, Brookline, MA); Pierre Cremieux and Denise Jarvinen, Brookline, MA; gift to NGA, 2010.',
  'classification': 'Photograph',
  'subclassification': '',
  'iiifthumburl': 'https://api.nga.gov/iiif/f4685ee0-1d01-4a40-9e16-6779e29e18ff/full/!200,200/0/default.jpg',
  'assistivetext': '',
  'sentences': ['Milton Rogovin, Buffalo, NY; David Knaus, Palm Springs, CA; (Morehouse Gallery, Brookline, MA); Pierre Cremieux and Denise Jarvinen, Brookline, MA; gift to NGA, 2010.'],
  'page_sentence_count_spacy': 1}]

In [7]:
df = pd.DataFrame(list_of_dicts)
df.describe().round(2)

,Unnamed: 0,objectid,page_sentence_count_spacy
count,109576.00,109576.00,109576.00
mean,57785.75,87573.21,1.02
std,33511.38,66784.27,2.25
min,1.00,1.00,0.00
25%,28565.75,29460.75,0.00
50%,58346.50,66717.50,1.00
75%,86213.25,145174.25,1.00
max,116240.00,228222.00,63.00


In [8]:
num_sentence_chunk_size = 10 

def split_list(input_list: list, 
               slice_size: int) -> list[list[str]]:
    return [input_list[i:i + slice_size] for i in range(0, len(input_list), slice_size)]

for item in tqdm(list_of_dicts):
    item["sentence_chunks"] = split_list(input_list=item["sentences"],
                                         slice_size=num_sentence_chunk_size)
    item["num_chunks"] = len(item["sentence_chunks"])

  0%|          | 0/109576 [00:00<?, ?it/s]

In [9]:
random.sample(list_of_dicts, k=1)

[{'Unnamed: 0': 77796,
  'objectid': 26381,
  'title': 'Silver Sugar Bowl',
  'beginyear': 1935.0,
  'endyear': 1942.0,
  'medium': 'graphite on paper',
  'attribution': 'Alfred Nason',
  'provenancetext': '',
  'classification': 'Index of American Design',
  'subclassification': 'Drawing',
  'iiifthumburl': 'https://api.nga.gov/iiif/84b57462-74f7-4e0d-a4ca-a5e81907be81/full/!200,200/0/default.jpg',
  'assistivetext': '',
  'sentences': [],
  'page_sentence_count_spacy': 0,
  'sentence_chunks': [],
  'num_chunks': 0}]

In [10]:
df = pd.DataFrame(list_of_dicts)
df.describe().round(2)

,Unnamed: 0,objectid,page_sentence_count_spacy,num_chunks
count,109576.00,109576.00,109576.00,109576.00
mean,57785.75,87573.21,1.02,0.55
std,33511.38,66784.27,2.25,0.53
min,1.00,1.00,0.00,0.00
25%,28565.75,29460.75,0.00,0.00
50%,58346.50,66717.50,1.00,1.00
75%,86213.25,145174.25,1.00,1.00
max,116240.00,228222.00,63.00,7.00


In [11]:
import re

pages_and_chunks = []
for item in tqdm(list_of_dicts):
    for sentence_chunk in item["sentence_chunks"]:
        chunk_dict = {}
        chunk_dict["number"] = item["Unnamed: 0"]
        
        joined_sentence_chunk = "".join(sentence_chunk).replace("  ", " ").strip()
        joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk)
        chunk_dict["sentence_chunk"] = joined_sentence_chunk

        chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
        chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4 
        
        pages_and_chunks.append(chunk_dict)

len(pages_and_chunks)

  0%|          | 0/109576 [00:00<?, ?it/s]

60226

In [12]:
random.sample(pages_and_chunks, k=1)

[{'number': 113125,
  'sentence_chunk': '(Enrico Cortona, Milan); purchased 2017 by NGA.',
  'chunk_char_count': 47,
  'chunk_word_count': 7,
  'chunk_token_count': 11.75}]

In [13]:
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

,number,chunk_char_count,chunk_word_count,chunk_token_count
count,60226.00,60226.00,60226.00,60226.00
mean,57244.48,193.70,31.22,48.42
std,35374.70,304.69,49.47,76.17
min,1.00,0.00,1.00,0.00
25%,27419.25,53.00,9.00,13.25
50%,47902.50,89.00,14.00,22.25
75%,95331.75,162.00,25.00,40.50
max,116240.00,2986.00,470.00,746.50


In [14]:
min_token_length = 30
for row in df[df["chunk_token_count"] <= min_token_length].sample(5).iterrows():
    print(f'Chunk token count: {row[1]["chunk_token_count"]} | Text: {row[1]["sentence_chunk"]}')

Chunk token count: 7.0 | Text: Anonymous gift to NGA, 1998.
Chunk token count: 12.0 | Text: (Jan Johnson, Montreal); purchased by NGA, 2005.
Chunk token count: 10.0 | Text: The artist, New York; gift to NGA, 1990.
Chunk token count: 12.5 | Text: (Frederick Mulder, London); purchased 2007 by NGA.
Chunk token count: 4.5 | Text: Gift to NGA, 1990.


In [15]:
pages_and_chunks_over_min_token_len = df[df["chunk_token_count"] > min_token_length].to_dict(orient="records")
pages_and_chunks_over_min_token_len[:2]

[{'number': 1,
  'sentence_chunk': 'Against a shiny gold background, a man with peach-colored skin nearly fills this tall, narrow, vertical panel above a row of twelve people, significantly smaller in scale, kneeling by his feet and looking up at the man, their hands together in prayer. The top of the panel comes to a point above the man’s head. His body faces us, and he looks at us with brown eyes under furrowed, dark eyebrows. His nose is long and straight, and his mouth is small with a rosy lower lip. He has a brown, trimmed beard, and his hair is receding. He holds up a gold-handled sword with a slate-gray blade in his right hand, on our left, while in his other hand, he cradles a thick black book with a gold-edged pages against his chest. The man wears a floor-length, celery-green tunic with a flowing, rose-pink robe over his left shoulder, on our right, and around his body. The robe is lined with tangerine orange and has a faint, patterned border. The toes of his sandaled feet st

In [16]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2", 
                                      device="cpu")

sentences = [
    "The Sentences Transformers library provides an easy and open-source way to create embeddings.",
    "Sentences can be embedded one by one or as a list of strings.",
    "Embeddings are one of the most powerful concepts in machine learning!",
    "Learn to use embeddings well and you'll be well on your way to being an AI engineer."
]

embeddings = embedding_model.encode(sentences)
embeddings_dict = dict(zip(sentences, embeddings))

for sentence, embedding in embeddings_dict.items():
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")

Sentence: The Sentences Transformers library provides an easy and open-source way to create embeddings.
Embedding: [-2.07983200e-02  3.03164814e-02 -2.01217812e-02  6.86484948e-02
 -2.55256109e-02 -8.47687945e-03 -2.07220146e-04 -6.32377192e-02
  2.81606615e-02 -3.33353654e-02  3.02633923e-02  5.30721210e-02
 -5.03526852e-02  2.62288619e-02  3.33313867e-02 -4.51577455e-02
  3.63044776e-02 -1.37122895e-03 -1.20171504e-02  1.14947017e-02
  5.04511073e-02  4.70856912e-02  2.11913809e-02  5.14606461e-02
 -2.03746762e-02 -3.58889364e-02 -6.67759334e-04 -2.94393804e-02
  4.95859124e-02 -1.05639528e-02 -1.52014270e-02 -1.31761038e-03
  4.48197536e-02  1.56023586e-02  8.60379373e-07 -1.21391879e-03
 -2.37978902e-02 -9.09328402e-04  7.34484056e-03 -2.53934250e-03
  5.23370542e-02 -4.68043610e-02  1.66214500e-02  4.71579805e-02
 -4.15599346e-02  9.01977066e-04  3.60278189e-02  3.42214108e-02
  9.68227237e-02  5.94829135e-02 -1.64984465e-02 -3.51249352e-02
  5.92518784e-03 -7.07923085e-04 -2.4103

In [17]:
single_sentence = "Yo! How cool are embeddings?"
single_embedding = embedding_model.encode(single_sentence)
print(f"Sentence: {single_sentence}")
print(f"Embedding:\n{single_embedding}")
print(f"Embedding size: {single_embedding.shape}")

Sentence: Yo! How cool are embeddings?
Embedding:
[-1.97447911e-02 -4.51076683e-03 -4.98486962e-03  6.55445009e-02
 -9.87673271e-03  2.72836164e-02  3.66426110e-02 -3.30218975e-03
  8.50077160e-03  8.24952219e-03 -2.28497945e-02  4.02430333e-02
 -5.75200319e-02  6.33691922e-02  4.43207175e-02 -4.49506640e-02
  1.25284679e-02 -2.52011549e-02 -3.55292894e-02  1.29559087e-02
  8.67024250e-03 -1.92917567e-02  3.55636678e-03  1.89505555e-02
 -1.47128431e-02 -9.39848460e-03  7.64176296e-03  9.62185021e-03
 -5.98922372e-03 -3.90168950e-02 -5.47824986e-02 -5.67458058e-03
  1.11644529e-02  4.08067293e-02  1.76319099e-06  9.15305968e-03
 -8.77257623e-03  2.39382815e-02 -2.32784431e-02  8.04999620e-02
  3.19177173e-02  5.12600737e-03 -1.47708422e-02 -1.62525252e-02
 -6.03213012e-02 -4.35689613e-02  4.51211631e-02 -1.79053508e-02
  2.63366681e-02 -3.47866938e-02 -8.89171846e-03 -5.47675565e-02
 -1.24372784e-02 -2.38606595e-02  8.33496973e-02  5.71241640e-02
  1.13328444e-02 -1.49594927e-02  9.2037

In [18]:
import torch
print("PyTorch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA Version:", torch.version.cuda)
    print("Device Name:", torch.cuda.get_device_name(0))


PyTorch Version: 2.2.2+cu121
CUDA Available: True
CUDA Version: 12.1
Device Name: NVIDIA GeForce GTX 1650


In [19]:
%%time

embedding_model.to("cuda") 

for item in tqdm(pages_and_chunks_over_min_token_len):
    item["embedding"] = embedding_model.encode(item["sentence_chunk"])

  0%|          | 0/22029 [00:00<?, ?it/s]

CPU times: total: 15min 32s
Wall time: 10min 56s


In [20]:
text_chunks = [item["sentence_chunk"] for item in pages_and_chunks_over_min_token_len]

In [21]:
import torch
torch.cuda.empty_cache()


In [22]:
import subprocess

def get_nvidia_smi_output():
    try:
        nvidia_smi_output = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE, check=True, text=True)
        return nvidia_smi_output.stdout
    except subprocess.CalledProcessError as e:
        return f"An error occurred: {e}"

print(get_nvidia_smi_output())


Thu Apr 18 07:52:11 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.78                 Driver Version: 551.78         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650      WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   83C    P8              4W /   50W |     538MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [23]:
import gc
gc.collect()


100

In [70]:
%%time

text_chunk_embeddings = embedding_model.encode(text_chunks,
                                               batch_size=200,
                                               convert_to_tensor=True)

text_chunk_embeddings

CPU times: total: 1h 41min 1s
Wall time: 2h 15min 36s


tensor([[ 0.0632,  0.0295, -0.0311,  ...,  0.0524,  0.0128, -0.0359],
        [ 0.0280, -0.0081, -0.0427,  ...,  0.0481,  0.0053, -0.0293],
        [ 0.0148,  0.0486, -0.0298,  ...,  0.0119,  0.0234, -0.0277],
        ...,
        [ 0.0342,  0.0233, -0.0030,  ...,  0.0203,  0.0126, -0.0532],
        [ 0.0507, -0.0269, -0.0125,  ...,  0.0073,  0.0126,  0.0156],
        [ 0.0386, -0.0405, -0.0064,  ...,  0.0548, -0.0108, -0.0062]],
       device='cuda:0')

In [73]:
import pandas as pd

cpu_embeddings = [embedding.cpu().numpy() for embedding in text_chunk_embeddings]

full_text_chunks_and_embeddings_df = pd.DataFrame(cpu_embeddings)

embeddings_df_save_path = "full_text_chunks_and_embeddings_df.csv"
full_text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index=False)


In [74]:
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_over_min_token_len)
embeddings_df_save_path = "text_chunks_and_embeddings_df.csv"
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index=False)

In [24]:
text_chunks_and_embedding_df_load = pd.read_csv("text_chunks_and_embeddings_df.csv")
text_chunks_and_embedding_df_load.head()

,number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,1,"Against a shiny gold background, a man with pe...",1099,200,274.75,[ 6.31980598e-02 2.95094457e-02 -3.11296210e-...
1,1,Eleven of the people gaze with heads tipped ba...,974,172,243.50,[ 2.79667582e-02 -8.14595819e-03 -4.26615328e-...
2,2,"(Stefano Bardini [1836-1922], Florence); sold ...",946,144,236.50,[ 1.47607662e-02 4.85922582e-02 -2.97773052e-...
3,3,Shown from the waist up on the other side of a...,960,172,240.00,[ 4.00882810e-02 3.80701087e-02 -4.90484247e-...
4,3,The wall is sage green and mustard yellow in t...,206,38,51.50,[ 4.50784341e-02 1.89006235e-02 -1.13143688e-...


In [25]:
import random

import torch
import numpy as np 
import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"

text_chunks_and_embedding_df = pd.read_csv("text_chunks_and_embeddings_df.csv")

text_chunks_and_embedding_df["embedding"] = text_chunks_and_embedding_df["embedding"].apply(lambda x: np.fromstring(x.strip("[]"), sep=" "))

pages_and_chunks = text_chunks_and_embedding_df.to_dict(orient="records")

embeddings = torch.tensor(np.array(text_chunks_and_embedding_df["embedding"].tolist()), dtype=torch.float32).to(device)
embeddings.shape

torch.Size([22029, 768])

In [26]:
text_chunks_and_embedding_df.head()

,number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,1,"Against a shiny gold background, a man with pe...",1099,200,274.75,"[0.0631980598, 0.0295094457, -0.031129621, 0.0..."
1,1,Eleven of the people gaze with heads tipped ba...,974,172,243.50,"[0.0279667582, -0.00814595819, -0.0426615328, ..."
2,2,"(Stefano Bardini [1836-1922], Florence); sold ...",946,144,236.50,"[0.0147607662, 0.0485922582, -0.0297773052, 0...."
3,3,Shown from the waist up on the other side of a...,960,172,240.00,"[0.040088281, 0.0380701087, -0.00490484247, 0...."
4,3,The wall is sage green and mustard yellow in t...,206,38,51.50,"[0.0450784341, 0.0189006235, -0.0113143688, 0...."


In [27]:
embeddings[0]

tensor([ 6.3198e-02,  2.9509e-02, -3.1130e-02,  8.4404e-02, -6.5375e-02,
         4.1338e-02,  5.3910e-02,  5.2898e-03,  2.4938e-02, -1.7048e-02,
        -1.0205e-02, -7.8306e-02,  7.0284e-02, -1.0468e-01,  4.2226e-02,
        -9.5233e-02, -9.7452e-03,  4.3040e-02, -2.7029e-02,  1.2393e-02,
        -3.0433e-02,  9.1391e-03,  7.7733e-03,  2.2135e-02,  9.5452e-03,
        -1.3414e-02, -1.3371e-02,  3.0397e-02, -2.9629e-02, -7.2319e-03,
        -3.1398e-02, -2.6732e-03, -3.9906e-02,  2.4617e-02,  2.3719e-06,
        -3.3410e-02,  3.2695e-02, -4.1489e-02,  4.1440e-02,  1.1045e-02,
         4.0233e-02, -1.5336e-02, -1.6260e-02, -4.1610e-02, -1.4377e-02,
         6.9530e-03, -1.7634e-02,  1.1386e-01,  3.7254e-02,  3.5056e-02,
         4.4813e-03, -4.9354e-02, -4.7782e-02, -1.8344e-02,  3.4325e-02,
         3.4031e-02,  2.0483e-02, -3.5128e-02,  7.4993e-03,  3.8380e-02,
        -4.3877e-02,  5.1151e-03, -3.6487e-02, -2.0284e-02,  2.1557e-03,
        -2.7187e-02,  2.1533e-02, -2.2342e-02,  1.4

In [28]:
from sentence_transformers import util, SentenceTransformer

embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2", 
                                      device=device) 

In [30]:

query = "Saint Paul and a Group of Worshippers"
print(f"Query: {query}")

query_embedding = embedding_model.encode(query, convert_to_tensor=True)

from time import perf_counter as timer

start_time = timer()
dot_scores = util.dot_score(a=query_embedding, b=embeddings)[0]
end_time = timer()

print(f"Time take to get scores on {len(embeddings)} embeddings: {end_time-start_time:.5f} seconds.")

top_results_dot_product = torch.topk(dot_scores, k=5)
top_results_dot_product

Query: Saint Paul and a Group of Worshippers
Time take to get scores on 22029 embeddings: 0.00063 seconds.


torch.return_types.topk(
values=tensor([0.5041, 0.4920, 0.4840, 0.4640, 0.4522], device='cuda:0'),
indices=tensor([19752, 16647, 12052, 12670, 16646], device='cuda:0'))

In [31]:
larger_embeddings = torch.randn(100*embeddings.shape[0], 768).to(device)
print(f"Embeddings shape: {larger_embeddings.shape}")

start_time = timer()
dot_scores = util.dot_score(a=query_embedding, b=larger_embeddings)[0]
end_time = timer()

print(f"Time take to get scores on {len(larger_embeddings)} embeddings: {end_time-start_time:.5f} seconds.")

Embeddings shape: torch.Size([2202900, 768])
Time take to get scores on 2202900 embeddings: 0.09021 seconds.


In [29]:
import torch
torch.cuda.empty_cache()


In [30]:
import gc
gc.collect()

7

In [31]:
import textwrap

def print_wrapped(text, wrap_length=80):
    wrapped_text = textwrap.fill(text, wrap_length)
    print(wrapped_text)

In [35]:
print(f"Query: '{query}'\n")
print("Results:")

for score, idx in zip(top_results_dot_product[0], top_results_dot_product[1]):
    print(f"Score: {score:.4f}")
    
    print("Text:")
    print_wrapped(pages_and_chunks[idx]["sentence_chunk"])
    
    print(f"Page number: {pages_and_chunks[idx]['number']}")
    print("\n")

Query: 'Saint Paul and a Group of Worshippers'

Results:
Score: 0.5041
Text:
On a ledge-like rocky outcropping, a man kneels and looks up at an angel while,
below and closer to us, three people sit or lie near the same man, who is shown
twice in this vertical painting. The people have light skin with a greenish
cast, though their cheeks are flushed, and they have flat, gold halos around
their heads. In both appearances, Jesus has long, wavy, amber-blond hair, a
beard, and a long straight nose. He wears a lapis-blue robe under a rose-pink
cloak, which are edged in gold. Closest to us, he stands near the lower left
corner of the composition, facing our right almost in profile. He holds one hand
to his chest and with the other, clasps the hand of a man kneeling in front of
him. That man, Saint Peter, wears a sky-blue robe and a canary-yellow cloak, and
he has a gray beard and hair. His brows are lowered and furrowed as he reaches
for and looks up at Jesus. Between Jesus and Saint Peter, a

In [33]:
import torch

def dot_product(vector1, vector2):
    return torch.dot(vector1, vector2)

def cosine_similarity(vector1, vector2):
    dot_product = torch.dot(vector1, vector2)

    
    norm_vector1 = torch.sqrt(torch.sum(vector1**2))
    norm_vector2 = torch.sqrt(torch.sum(vector2**2))

    return dot_product / (norm_vector1 * norm_vector2)

vector1 = torch.tensor([1, 2, 3], dtype=torch.float32)
vector2 = torch.tensor([1, 2, 3], dtype=torch.float32)
vector3 = torch.tensor([4, 5, 6], dtype=torch.float32)
vector4 = torch.tensor([-1, -2, -3], dtype=torch.float32)

print("Dot product between vector1 and vector2:", dot_product(vector1, vector2))
print("Dot product between vector1 and vector3:", dot_product(vector1, vector3))
print("Dot product between vector1 and vector4:", dot_product(vector1, vector4))

print("Cosine similarity between vector1 and vector2:", cosine_similarity(vector1, vector2))
print("Cosine similarity between vector1 and vector3:", cosine_similarity(vector1, vector3))
print("Cosine similarity between vector1 and vector4:", cosine_similarity(vector1, vector4))

Dot product between vector1 and vector2: tensor(14.)
Dot product between vector1 and vector3: tensor(32.)
Dot product between vector1 and vector4: tensor(-14.)
Cosine similarity between vector1 and vector2: tensor(1.0000)
Cosine similarity between vector1 and vector3: tensor(0.9746)
Cosine similarity between vector1 and vector4: tensor(-1.0000)


In [37]:
from time import perf_counter as timer
def retrieve_relevant_resources(query: str,
                                embeddings: torch.tensor,
                                model: SentenceTransformer=embedding_model,
                                n_resources_to_return: int=5,
                                print_time: bool=True):
    query_embedding = model.encode(query, 
                                   convert_to_tensor=True) 

    start_time = timer()
    dot_scores = util.dot_score(query_embedding, embeddings)[0]
    end_time = timer()

    if print_time:
        print(f"[INFO] Time taken to get scores on {len(embeddings)} embeddings: {end_time-start_time:.5f} seconds.")

    scores, indices = torch.topk(input=dot_scores, 
                                 k=n_resources_to_return)

    return scores, indices

def print_top_results_and_scores(query: str,
                                 embeddings: torch.tensor,
                                 pages_and_chunks: list[dict]=pages_and_chunks,
                                 n_resources_to_return: int=5):
    
    scores, indices = retrieve_relevant_resources(query=query,
                                                  embeddings=embeddings,
                                                  n_resources_to_return=n_resources_to_return)
    
    print(f"Query: {query}\n")
    print("Results:")
    for score, index in zip(scores, indices):
        print(f"Score: {score:.4f}")
        print_wrapped(pages_and_chunks[index]["sentence_chunk"])
        print(f"Page number: {pages_and_chunks[index]['number']}")
        print("\n")

In [38]:
import gc
gc.collect()

800

In [39]:
query = "An Old Woman Dozing over a Book"

scores, indices = retrieve_relevant_resources(query=query,
                                              embeddings=embeddings)
scores, indices

[INFO] Time taken to get scores on 22029 embeddings: 0.00052 seconds.


(tensor([0.4630, 0.3793, 0.3609, 0.3481, 0.3449], device='cuda:0'),
 tensor([    3, 14943, 21712, 21691, 16129], device='cuda:0'))

In [40]:
print_top_results_and_scores(query=query,
                             embeddings=embeddings)

[INFO] Time taken to get scores on 22029 embeddings: 0.00009 seconds.
Query: An Old Woman Dozing over a Book

Results:
Score: 0.4630
Shown from the waist up on the other side of a strawberry-red table, an elderly,
pale-skinned woman sits facing us, sleeping with her head propped on her hand in
this vertical painting. She is brightly lit from our upper left, which creates
deep shadows across her face, and behind and around her. The woman rests her
cheek against her closed left fist, to our right. That elbow is propped on a
gold and amber-yellow cushion tucked into the corner where the table meets the
wall to our right. Her brows are slightly furrowed over deep-set eyes. She has
loose jowls, thin lips, and wrinkles line the corners of her eyes and her mouth.
Almost lost in shadow, her other hand rests on the large book lying open before
her. She wears a black coat draped over a parchment-white garment. Her hair is
covered by a ginger-brown and black striped veil that hangs to her shoulde

In [41]:
import torch
gpu_memory_bytes = torch.cuda.get_device_properties(0).total_memory
gpu_memory_gb = round(gpu_memory_bytes / (2**30))
print(f"Available GPU memory: {gpu_memory_gb} GB")

Available GPU memory: 4 GB


In [42]:
model_id = "google/gemma-2b-it"
use_quantization_config = True

In [43]:
from huggingface_hub import notebook_login
notebook_login()

In [44]:
import gc
gc.collect()

0

In [45]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.utils import is_flash_attn_2_available 

from transformers import BitsAndBytesConfig
quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                         bnb_4bit_compute_dtype=torch.float16)

if (is_flash_attn_2_available()) and (torch.cuda.get_device_capability(0)[0] >= 8):
  attn_implementation = "flash_attention_2"
else:
  attn_implementation = "sdpa"
print(f"[INFO] Using attention implementation: {attn_implementation}")

model_id = model_id
print(f"[INFO] Using model_id: {model_id}")

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_id)

llm_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_id, 
                                                 torch_dtype=torch.float16, 
                                                 quantization_config=quantization_config if use_quantization_config else None,
                                                 low_cpu_mem_usage=True,
                                                 attn_implementation=attn_implementation)

if not use_quantization_config:
    llm_model.to("cuda")

[INFO] Using attention implementation: sdpa
[INFO] Using model_id: google/gemma-2b-it


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [46]:
llm_model

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
     

In [47]:
def get_model_num_params(model: torch.nn.Module):
    return sum([param.numel() for param in model.parameters()])

get_model_num_params(llm_model)

1515268096

In [48]:
def get_model_mem_size(model: torch.nn.Module):

    mem_params = sum([param.nelement() * param.element_size() for param in model.parameters()])
    mem_buffers = sum([buf.nelement() * buf.element_size() for buf in model.buffers()])

    model_mem_bytes = mem_params + mem_buffers
    model_mem_mb = model_mem_bytes / (1024**2)
    model_mem_gb = model_mem_bytes / (1024**3)

    return {"model_mem_bytes": model_mem_bytes,
            "model_mem_mb": round(model_mem_mb, 2),
            "model_mem_gb": round(model_mem_gb, 2)}

get_model_mem_size(llm_model)

{'model_mem_bytes': 2106740736, 'model_mem_mb': 2009.14, 'model_mem_gb': 1.96}

In [105]:
input_text = "Give details about the painting Saint Ildefonso"
print(f"Input text:\n{input_text}")

dialogue_template = [
    {"role": "user",
     "content": input_text}
]

prompt = tokenizer.apply_chat_template(conversation=dialogue_template,
                                       tokenize=False,
                                       add_generation_prompt=True)
print(f"\nPrompt (formatted):\n{prompt}")

Input text:
Give details about the painting Saint Ildefonso

Prompt (formatted):
<bos><start_of_turn>user
Give details about the painting Saint Ildefonso<end_of_turn>
<start_of_turn>model



In [106]:
%%time

input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
print(f"Model input (tokenized):\n{input_ids}\n")


outputs = llm_model.generate(**input_ids,
                             max_new_tokens=256)
print(f"Model output (tokens):\n{outputs[0]}\n")

Model input (tokenized):
{'input_ids': tensor([[    2,     2,   106,  1645,   108, 23136,  4691,  1105,   573, 11825,
         10606,  4626,  1293, 27306,   107,   108,   106,  2516,   108]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}

Model output (tokens):
tensor([     2,      2,    106,   1645,    108,  23136,   4691,   1105,    573,
         11825,  10606,   4626,   1293,  27306,    107,    108,    106,   2516,
           108,    688,   4883,  66058,  10606,   4626,   1293,  27306,    109,
           688,  29452,  66058,  15256,  47643,    109,    688,   2504,  66058,
        235248, 235274, 235318, 235276, 235308,    109,    688,  28484,  66058,
         12799,    611,  13381,    109,    688,   5360,  66058,   4094,  17627,
           576,   5936, 235269,  15500, 235269,  14034,    109,    688,   3602,
         66058,    109, 235287,    714,  11825,  75231,    573,   4318,   1185,
        

In [107]:
outputs_decoded = tokenizer.decode(outputs[0])
print(f"Model output (decoded):\n{outputs_decoded}\n")

Model output (decoded):
<bos><bos><start_of_turn>user
Give details about the painting Saint Ildefonso<end_of_turn>
<start_of_turn>model
**Title:** Saint Ildefonso

**Artist:** Juan Rodriguez

**Date:** 1605

**Medium:** Oil on canvas

**Location:** National Gallery of Art, Madrid, Spain

**Description:**

* The painting depicts the moment when Saint Ildefonso, a Spanish bishop and martyr, is about to be beheaded.
* The scene is dramatic and emotionally charged, with the artist capturing the anguish and finality of the event.
* The painting is a masterpiece of Spanish Baroque art and a powerful symbol of religious devotion and martyrdom.

**Background:**

* The painting was commissioned by the Marquis of Villanuegra, a prominent Spanish nobleman.
* It was part of a series of religious paintings commissioned by the Marquis to commemorate the life and miracles of Saint Ildefonso.
* The series also included paintings of the saint's life, his martyrdom, and his sainthood.

**Style:**

* The

In [108]:
print(f"Input text: {input_text}\n")
print(f"Output text:\n{outputs_decoded.replace(prompt, '').replace('<bos>', '').replace('<eos>', '')}")

Input text: Give details about the painting Saint Ildefonso

Output text:
**Title:** Saint Ildefonso

**Artist:** Juan Rodriguez

**Date:** 1605

**Medium:** Oil on canvas

**Location:** National Gallery of Art, Madrid, Spain

**Description:**

* The painting depicts the moment when Saint Ildefonso, a Spanish bishop and martyr, is about to be beheaded.
* The scene is dramatic and emotionally charged, with the artist capturing the anguish and finality of the event.
* The painting is a masterpiece of Spanish Baroque art and a powerful symbol of religious devotion and martyrdom.

**Background:**

* The painting was commissioned by the Marquis of Villanuegra, a prominent Spanish nobleman.
* It was part of a series of religious paintings commissioned by the Marquis to commemorate the life and miracles of Saint Ildefonso.
* The series also included paintings of the saint's life, his martyrdom, and his sainthood.

**Style:**

* The painting is an example of the High Renaissance style, with it

In [109]:
questions = [
    "Give details about the painting An Old Woman Dozing over a Book",
    "Give details about the painting Saint Ildefonso"
]
query_list = questions

In [110]:
import random
query = random.choice(query_list)

print(f"Query: {query}")

scores, indices = retrieve_relevant_resources(query=query,
                                              embeddings=embeddings)
scores, indices

Query: Give details about the painting An Old Woman Dozing over a Book
[INFO] Time taken to get scores on 22029 embeddings: 0.00012 seconds.


(tensor([0.7234, 0.7213, 0.6961, 0.6955, 0.6833], device='cuda:0'),
 tensor([21712,     3, 18578, 14500,  1631], device='cuda:0'))

In [111]:
def prompt_formatter(query: str, 
                     context_items: list[dict]) -> str:
    
    context = "- " + "\n- ".join([item["sentence_chunk"] for item in context_items])

    base_prompt = """Based on the following context items, please answer the query.
Give yourself room to think by extracting relevant passages from the context before answering the query.
Don't return the thinking, only return the answer.
Make sure your answers are as explanatory as possible.
Use the following examples as reference for the ideal answer style.
\nExample 1:
Query: What are the fat-soluble vitamins?
Answer: The fat-soluble vitamins include Vitamin A, Vitamin D, Vitamin E, and Vitamin K. These vitamins are absorbed along with fats in the diet and can be stored in the body's fatty tissue and liver for later use. Vitamin A is important for vision, immune function, and skin health. Vitamin D plays a critical role in calcium absorption and bone health. Vitamin E acts as an antioxidant, protecting cells from damage. Vitamin K is essential for blood clotting and bone metabolism.
\nExample 2:
Query: What are the causes of type 2 diabetes?
Answer: Type 2 diabetes is often associated with overnutrition, particularly the overconsumption of calories leading to obesity. Factors include a diet high in refined sugars and saturated fats, which can lead to insulin resistance, a condition where the body's cells do not respond effectively to insulin. Over time, the pancreas cannot produce enough insulin to manage blood sugar levels, resulting in type 2 diabetes. Additionally, excessive caloric intake without sufficient physical activity exacerbates the risk by promoting weight gain and fat accumulation, particularly around the abdomen, further contributing to insulin resistance.
\nExample 3:
Query: What is the importance of hydration for physical performance?
Answer: Hydration is crucial for physical performance because water plays key roles in maintaining blood volume, regulating body temperature, and ensuring the transport of nutrients and oxygen to cells. Adequate hydration is essential for optimal muscle function, endurance, and recovery. Dehydration can lead to decreased performance, fatigue, and increased risk of heat-related illnesses, such as heat stroke. Drinking sufficient water before, during, and after exercise helps ensure peak physical performance and recovery.
\nNow use the following context items to answer the user query:
{context}
\nRelevant passages: <extract relevant passages from the context here>
User query: {query}
Answer:"""
  
    base_prompt = base_prompt.format(context=context, query=query)

    dialogue_template = [
        {"role": "user",
        "content": base_prompt}
    ]

    prompt = tokenizer.apply_chat_template(conversation=dialogue_template,
                                          tokenize=False,
                                          add_generation_prompt=True)
    return prompt

In [112]:
query = random.choice(query_list)
print(f"Query: {query}")

scores, indices = retrieve_relevant_resources(query=query,
                                              embeddings=embeddings)
    
context_items = [pages_and_chunks[i] for i in indices]

prompt = prompt_formatter(query=query,
                          context_items=context_items)
print(prompt)

Query: Give details about the painting An Old Woman Dozing over a Book
[INFO] Time taken to get scores on 22029 embeddings: 0.00008 seconds.
<bos><start_of_turn>user
Based on the following context items, please answer the query.
Give yourself room to think by extracting relevant passages from the context before answering the query.
Don't return the thinking, only return the answer.
Make sure your answers are as explanatory as possible.
Use the following examples as reference for the ideal answer style.

Example 1:
Query: What are the fat-soluble vitamins?
Answer: The fat-soluble vitamins include Vitamin A, Vitamin D, Vitamin E, and Vitamin K. These vitamins are absorbed along with fats in the diet and can be stored in the body's fatty tissue and liver for later use. Vitamin A is important for vision, immune function, and skin health. Vitamin D plays a critical role in calcium absorption and bone health. Vitamin E acts as an antioxidant, protecting cells from damage. Vitamin K is essent

In [125]:
import gc
gc.collect()

3602

In [124]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


In [126]:
import torch
torch.cuda.empty_cache()


In [116]:
%%time

input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

outputs
with torch.no_grad():
    outputs = llm_model.generate(**input_ids,
                                  temperature=0.7,
                                  do_sample=True,
                                  max_new_tokens=250)
# # Generate an output of tokens
# outputs = llm_model.generate(**input_ids,
#                              temperature=0.7,
#                              do_sample=True,
#                              max_new_tokens=10)

# Turn the output tokens into text
output_text = tokenizer.decode(outputs[0])

print(f"Query: {query}")
print(f"RAG answer:\n{output_text.replace(prompt, '')}")

Query: Give details about the painting An Old Woman Dozing over a Book
RAG answer:
<bos>Sure, here are the relevant passages from the context:

**Passage 1:**
- The painting depicts an older woman sitting in a chair, reading a book.
- Her pale skin and long, flowing dress with a wide skirt create a sense of elegance and grace.
- The room is dimly lit, highlighting the woman's pale face and the shadows cast by the furniture.

**Passage 2:**
- The woman is depicted asleep, with her head resting on her hand and her eyes closed.
- The portrait emphasizes her age and the shadows and light that create a sense of mystery and serenity.
- The painting highlights her pale skin, wrinkles, and the book she is reading, suggesting a quiet and contemplative personality.

**Passage 3:**
- The painting depicts an elderly woman sitting in a darkened room, reading.
- Her pale skin, half-closed eyes, and chin resting on a book suggest a contemplative and thoughtful demeanor.
- The single candle casts a wa

In [127]:
def ask(query, 
        temperature=0.7,
        max_new_tokens=512,
        format_answer_text=True, 
        return_answer_only=True):

    scores, indices = retrieve_relevant_resources(query=query,
                                                  embeddings=embeddings)
    
    context_items = [pages_and_chunks[i] for i in indices]

    for i, item in enumerate(context_items):
        item["score"] = scores[i].cpu()
        
    prompt = prompt_formatter(query=query,
                              context_items=context_items)
    
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

    outputs = llm_model.generate(**input_ids,
                                 temperature=temperature,
                                 do_sample=True,
                                 max_new_tokens=max_new_tokens)
    
    output_text = tokenizer.decode(outputs[0])

    if format_answer_text:
        output_text = output_text.replace(prompt, "").replace("<bos>", "").replace("<eos>", "").replace("Sure, here is the answer to the user query:\n\n", "")

    if return_answer_only:
        return output_text
    
    return output_text, context_items

In [129]:
query = random.choice(query_list)
print(f"Query: {query}")

answer, context_items = ask(query=query, 
                            temperature=0.7,
                            max_new_tokens=512,
                            return_answer_only=False)

print(f"Answer:\n")
print_wrapped(answer)
print(f"Context items:")
context_items

Query: Give details about the painting An Old Woman Dozing over a Book
[INFO] Time taken to get scores on 22029 embeddings: 0.00009 seconds.
Answer:

**Relevant passages:**  * The painting depicts an older woman sitting in a
chair, reading a book. * Her pale skin, rosy facial features, and the folds in
her dress suggest an air of serenity and vulnerability. * The painting portrays
light and shadow effectively, highlighting the woman's facial features and the
room's shadows. * The woman's posture suggests a hint of discomfort or
weariness, as she is leaning back slightly. * The painting captures the quietude
and stillness of a peaceful afternoon moment.
Context items:


[{'number': 116012,
  'sentence_chunk': 'An older woman wearing a long black dress sits reading a book in front of a younger woman wearing cream white, who sits on a couch just beyond, in this vertical portrait painting. The scene is loosely painted with some visible brushstrokes, especially in the furniture, clothing, and room. Both women have pale skin and wear dresses with long sleeves and skirts. To our right, the older woman sits in a chair with a curved back, which runs parallel to the right edge of the canvas. The full skirt of her black dress angles down across the picture, almost spanning the bottom edge of the composition. Her gray hair is pulled back and covered by a black cloth or veil, which falls to her shoulder and blends with her dress. The collar is lined with a white ruffle, and white lace cuffs encircle her wrists. Her face is painted with rosy tones. She looks down at her book with dark eyes, and her rose-pink lips are closed. She wears a gold ring with a light ston